In [2]:
"""
Feature generation script (cleaned + labeled)

Inputs expected (per UniProt ID):
- ../data/whole_proteome_uniprot_id.xlsx                (Sequence, Entry Name, From->uniprot_ids)
- ../data/49_properties_normalizedValues.csv            (physicochemical properties; normalized)
- ../data/49_properties_numerical_Values.csv            (physicochemical properties; numeric)
- ../data/463_unique_numerical_properties.csv           (additional numeric properties)
- ../data/aaindex_square_diagonal_properties.csv        (AAindex mutation matrices; wild_mut index)
- ../data/{uniprot_id}.pssm                             (PSSM file)
- ../data/{uniprot_id}.csv                              (AACon conservation features)
- ../data/{uniprot_id}_IUPred.csv                        (disorder)
- ../data/{uniprot_id}_residue_depth.csv                 (residue depth)
- ../data/{uniprot_id}_plDDT.out                         (plDDT / B-factor per residue)
- ../data/{uniprot_id}.out                               (DSSP-like output with SS, ASA columns)
- ../data/{uniprot_id}_network.csv                       (network centrality features)
- ../data/structure_specific_interactions/{uniprot_id}_*.csv (interaction/contact files)

Output:
- ../complete_dataset_with_features.csv
"""

import random
import re
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

# -----------------------------
# CONFIG / INPUTS
# -----------------------------
INPUT_FILE = "../input_data/input.csv"

input_df = pd.read_csv(INPUT_FILE)

# sanity check
required_cols = {"uniprot_id", "mutation"}
if not required_cols.issubset(input_df.columns):
    raise ValueError("input.csv must contain columns: uniprot_id, mutation")
BASE_DIR = "../data"
OUT_CSV = "../output_data/complete_dataset_with_features.csv"

AA3_TO_AA1 = {
    "ALA": "A", "ARG": "R", "ASN": "N", "ASP": "D", "CYS": "C",
    "GLN": "Q", "GLU": "E", "GLY": "G", "HIS": "H", "ILE": "I",
    "LEU": "L", "LYS": "K", "MET": "M", "PHE": "F", "PRO": "P",
    "SER": "S", "THR": "T", "TRP": "W", "TYR": "Y", "VAL": "V"
}

MOTIFS = ['nM', 'Mc', 'n_M', 'M_c', 'n__M', 'M__c', 'tri']

AACON_HEADER = [
    'mut_pos', 'KABAT', 'JORES', 'SCHNEIDER', 'SHENKIN', 'GERSTEIN',
    'TAYLOR_GAPS', 'TAYLOR_NO_GAPS', 'VELIBIL', 'KARLIN', 'ARMON',
    'THOMPSON', 'NOT_LANCET', 'MIRNY', 'WILLIAMSON', 'LANDGRAF',
    'SANDER', 'VALDAR', 'SMERFS'
]

CONTACT_FILES = [
    "_aroaro.csv", "_arosul.csv", "_cationpi.csv", "_disulphide.csv",
    "_hbond_main_main.csv", "_hbond_main_side.csv", "_hbond_side_side.csv",
    "_hydrophobic.csv", "_ionic.csv"
]


# -----------------------------
# LOAD REFERENCE TABLES
# -----------------------------
uniprot_seq = pd.read_excel(f"{BASE_DIR}/whole_proteome_uniprot_id.xlsx", engine="openpyxl")
uniprot_seq.rename({"From": "uniprot_ids"}, axis=1, inplace=True)
uniprot_seq["gene"] = uniprot_seq["Entry Name"].apply(lambda x: x.split("_")[0])

physchem_norm = pd.read_csv(f"{BASE_DIR}/49_properties_normalizedValues.csv")
physchem_num = pd.read_csv(f"{BASE_DIR}/49_properties_numerical_Values.csv")
extra_463 = pd.read_csv(f"{BASE_DIR}/463_unique_numerical_properties.csv")

# NOTE: This creates a numeric-property table with the same columns as physchem_num
physchem_num_all = pd.concat([physchem_norm, extra_463[physchem_num.columns]], ignore_index=True)

mutation_matrices = pd.read_csv(
    f"{BASE_DIR}/aaindex_square_diagonal_properties.csv",
    keep_default_na=False
)
mutation_matrices.set_index("wild_mut", inplace=True)


# -----------------------------
# HELPER FUNCTIONS
# -----------------------------
def get_sequence_and_gene(uniprot_id: str):
    seq = next(iter(uniprot_seq.loc[uniprot_seq["uniprot_ids"] == uniprot_id, "Sequence"]))
    gene = next(iter(uniprot_seq.loc[uniprot_seq["uniprot_ids"] == uniprot_id, "gene"]))
    return seq, gene


def window_13mer(sequence: str, pos_1based: int) -> str:
    """13-aa window centered on position; handles ends."""
    pos0 = pos_1based - 1
    left = max(0, pos0 - 6)
    right = min(len(sequence), pos0 + 7)
    return sequence[left:right]


def safe_get(series, default=0):
    try:
        return next(iter(series))
    except StopIteration:
        return default


def compute_dipeptide_motifs(sequence: str, pos_1based: int, wild: str):
    """Motifs used for odds-ratio lookup."""
    pos0 = pos_1based - 1

    # N-terminal neighbors
    n1 = sequence[pos0 - 1] if pos0 - 1 >= 0 else '-'
    n2 = sequence[pos0 - 2] if pos0 - 2 >= 0 else '-'
    n4 = sequence[pos0 - 4] if pos0 - 4 >= 0 else '-'

    # C-terminal neighbors
    c1 = sequence[pos0 + 1] if pos0 + 1 < len(sequence) else '-'
    c2 = sequence[pos0 + 2] if pos0 + 2 < len(sequence) else '-'
    c0 = sequence[pos0 + 0] if pos0 < len(sequence) else '-'

    motifs = {}
    motifs["nM"] = n1 + wild
    motifs["Mc"] = wild + c0
    motifs["tri"] = n1 + wild + c0
    motifs["n_M"] = n2 + "*" + wild
    motifs["M_c"] = wild + "*" + c1
    motifs["n__M"] = (n4 + "**" + wild) if n4 != '-' else ('-' + wild)
    motifs["M__c"] = wild + "**" + (c2 if c2 != '-' else c1)

    return motifs


def add_physchem_features(dict_prop, wild, mut, win13):
    """
    Feature group: Physicochemical properties
    - site values (wild)
    - window mean (13-mer)
    - diff-like features
    """
    # window means across residues (ignoring non-letters)
    denom = sum(c.isalpha() for c in win13)
    a = sum(physchem_norm[win13[k]] for k in range(len(win13))) / denom
    b = sum(physchem_num_all[win13[k]] for k in range(len(win13))) / denom

    j = 0
    for prop_name in physchem_norm["index"]:
        # normalized
        wild_norm = physchem_norm.loc[physchem_norm["index"] == prop_name, wild].tolist()[0]
        mut_norm = physchem_norm.loc[physchem_norm["index"] == prop_name, mut].tolist()[0]
        dict_prop[f"{prop_name}_normalize_site_value"] = wild_norm
        dict_prop[f"{prop_name}_normalize"] = a[j]
        dict_prop[f"{prop_name}_normalize_diff"] = a[j] - mut_norm + wild_norm

        # numeric (FIX: original code accidentally used "pK'" row for everything)
        wild_num = physchem_num_all.loc[physchem_num_all["index"] == prop_name, wild].tolist()[0]
        mut_num = physchem_num_all.loc[physchem_num_all["index"] == prop_name, mut].tolist()[0]
        dict_prop[f"{prop_name}_numeric_site_value"] = wild_num
        dict_prop[f"{prop_name}_numeric_values"] = b[j]
        dict_prop[f"{prop_name}_numeric_diff"] = b[j] - mut_num + wild_num

        j += 1

    # Feature group: amino-acid class composition in 13-mer window
    dict_prop["neg_charge"] = len(re.findall("[DE]", win13))
    dict_prop["polar"] = len(re.findall("[NQSTP]", win13))
    dict_prop["aromatic"] = len(re.findall("[YFW]", win13))
    dict_prop["S_containing"] = len(re.findall("[CM]", win13))
    dict_prop["aliphatic"] = len(re.findall("[GALIV]", win13))


def add_mutation_matrix_features(dict_prop, sequence, pos, wild, mut):
    """
    Feature group: AAIndex mutation matrices
    - N-terminal context (mut_prop)
    - C-terminal context (lowercase mut_prop)
    """
    pos0 = pos - 1

    # N-terminal dipeptide: (wild + previous residue)
    prev_res = sequence[pos0 - 1] if pos0 - 1 >= 0 else "-"
    n_wild = wild + prev_res
    n_mut = mut + prev_res

    # C-terminal dipeptide: (wild + next residue)
    next_res = sequence[pos0 + 1] if pos0 + 1 < len(sequence) else "-"
    c_wild = wild + next_res
    c_mut = mut + next_res

    property_n = mutation_matrices.loc[n_mut] - mutation_matrices.loc[n_wild]
    try:
        property_c = mutation_matrices.loc[c_mut] - mutation_matrices.loc[c_wild]
    except KeyError:
        property_c = mutation_matrices.loc["AG"] - mutation_matrices.loc["AG"]

    for mut_prop in property_n.index:
        dict_prop[mut_prop] = property_n[mut_prop]
        dict_prop[mut_prop.lower()] = property_c[mut_prop]


def add_odds_ratio_features(dict_prop, motifs):
    """
    Feature group: Motif odds-ratio (from Excel sheets per motif)
    Creates:
    - {motif}_coded_odds
    - {motif}_odds_ratio
    """
    for motif in MOTIFS:
        x = pd.read_excel(f"{BASE_DIR}/BRCA_odds_ratio.xlsx", sheet_name=motif)
        entry = motifs[motif]
        row = x[x[motif] == entry]

        odd = safe_get(row["odd_ratio"], default=0)

        # Coded odds bins
        conditions = [
            (row["odd_ratio"] >= 1.2) | (row["odd_ratio"] == "inf"),
            (row["odd_ratio"] < 1.2) & (row["odd_ratio"] >= 0.9),
            (row["odd_ratio"] < 0.9),
        ]
        code_vals = [1, 3, 2]
        dict_prop[f"{motif}_coded_odds"] = safe_get(np.select(conditions, code_vals), default=0)
        dict_prop[f"{motif}_odds_ratio"] = odd


def add_network_features(dict_prop, df_network, wild, pos):
    """
    Feature group: Structural network centrality
    """
    xx = df_network[(df_network["Wild"] == wild) & (df_network["pos"] == pos)]
    if len(xx) > 0:
        dict_prop["Degree_centrality"] = safe_get(xx["Degree centrality"], 0)
        dict_prop["Closeness_centrality"] = safe_get(xx["Closeness centrality"], 0)
        dict_prop["betweenness_centrality"] = safe_get(xx["Betweenness centrality"], 0)
        dict_prop["eigenvector_centrality"] = safe_get(xx["Eigenvector centrality"], 0)
    else:
        dict_prop["Degree_centrality"] = 0
        dict_prop["Closeness_centrality"] = 0
        dict_prop["betweenness_centrality"] = 0
        dict_prop["eigenvector_centrality"] = 0


def add_pssm_features(dict_prop, f4_lines, pos, wild, mut):
    """
    Feature group: PSSM / conservation
    """
    # PSSM line indexing: original code uses f4[pos] (1-basedish). Keep as-is to match their files.
    cols = f4_lines[pos].strip().split()

    aa_scores = cols[2:22]  # 20 AAs
    aa_order = list("ARNDCQEGHILKMFPSTWYV")

    score_map = dict(zip(aa_order, aa_scores))
    score_map = {k: int(v) for k, v in score_map.items()}

    dict_prop["pssm_score1"] = score_map.get(wild, 0)
    dict_prop["pssm_score2"] = sum(score_map.values()) / 20.0
    dict_prop["pssm_score3"] = score_map.get(mut, 0) - score_map.get(wild, 0)

    # Conservation: original code slices at char 90; keep behavior
    cons = f4_lines[pos].strip("\n")[90:].split()[21:22]
    dict_prop["conservation"] = float(cons[0]) if len(cons) else 0.0


def add_aacon_features(dict_prop, f_aacon, pos):
    """
    Feature group: AACon conservation features (per residue)
    """
    mm = f_aacon.iloc[pos - 1]
    for item in AACON_HEADER:
        dict_prop[item] = mm[item]


def add_disorder_depth_plddt_dssp(dict_prop, f_disorder, f_depth, f_plddt, f_dssp, pos):
    """
    Feature group: Disorder + structure features
    - disorder (IUPred)
    - DSSP secondary structure + ASA
    - plDDT
    - residue depth
    """
    dict_prop["disorder"] = safe_get(
        f_disorder.loc[f_disorder["Pos"] == pos, "IUPRED SCORE"],
        default=0
    )

    # DSSP and other structure files: if residue missing, use averages/random SS
    if pos > len(f_dssp):
        dict_prop["sec_strc"] = random.choice(list("BEGHIST-"))
        dict_prop["ASA"] = float(np.average(f_dssp["ASA"])) if len(f_dssp) else 0
        dict_prop["plDDT"] = float(np.average(f_plddt["Avg. B-factor"])) if len(f_plddt) else 0
        dict_prop["f_residue_depth"] = float(np.average(f_depth["depth"])) if len(f_depth) else 0
    else:
        dict_prop["sec_strc"] = f_dssp.iloc[pos - 1]["SS"]
        dict_prop["ASA"] = f_dssp.iloc[pos - 1]["ASA"]
        dict_prop["plDDT"] = safe_get(
            f_plddt.loc[f_plddt["Residue"] == pos, "Avg. B-factor"],
            default=0
        )
        dict_prop["f_residue_depth"] = safe_get(
            f_depth.loc[f_depth["pos"] == pos, "depth"],
            default=0
        )


def add_structure_interaction_counts(dict_prop, uniprot_id, pos):
    """
    Feature group: Structure-specific interaction/contact counts
    Each file contributes one count feature keyed by interaction type.
    """
    for suffix in CONTACT_FILES:
        f_type = f"{uniprot_id}{suffix}"
        df_bonds = pd.read_csv(f"{BASE_DIR}/structure_specific_interactions/{f_type}")

        # Normalize residue labels and positions
        df_bonds["Res1"] = df_bonds["RES1 "].apply(lambda x: AA3_TO_AA1[x.strip()])
        df_bonds["Res2"] = df_bonds[" RES2 "].apply(lambda x: AA3_TO_AA1[x.strip()])
        df_bonds.rename({" idRES1 ": "pos1", " idRES2 ": "pos2"}, axis=1, inplace=True)

        interaction_name = f_type.split("_")[-1].split(".")[0]
        dict_prop[interaction_name] = int((df_bonds["pos1"] == pos).sum())


# -----------------------------
# MAIN: BUILD FEATURE TABLE
# -----------------------------
all_rows = []

for _, row in input_df.iterrows():
    uniprot_id = row["uniprot_id"]
    mutation = row["mutation"]

    wild = mutation[0]
    mut = mutation[-1]
    pos = int(mutation[1:-1])  # 1-based residue index

    sequence, gene = get_sequence_and_gene(uniprot_id)

    # Basic validation: position exists and wild matches sequence at that position
    if not (1 <= pos <= len(sequence)) or sequence[pos - 1] != wild:
        # Skip invalid entries (instead of silently doing nothing)
        continue

    win13 = window_13mer(sequence, pos)
    motifs = compute_dipeptide_motifs(sequence, pos, wild)

    # Load per-protein auxiliary files
    f4_lines = open(f"{BASE_DIR}/{uniprot_id}.pssm", "r").readlines()[2:]
    f_aacon = pd.read_csv(f"{BASE_DIR}/{uniprot_id}.csv", skiprows=1, names=AACON_HEADER)
    f_disorder = pd.read_csv(f"{BASE_DIR}/{uniprot_id}_IUPred.csv")
    f_depth = pd.read_csv(f"{BASE_DIR}/{uniprot_id}_residue_depth.csv")
    f_plddt = pd.read_csv(f"{BASE_DIR}/{uniprot_id}_plDDT.out")
    f_dssp = pd.read_csv(f"{BASE_DIR}/{uniprot_id}.out")
    df_network = pd.read_csv(f"{BASE_DIR}/{uniprot_id}_network.csv")

    # -----------------------------
    # Feature dictionary (one row)
    # -----------------------------
    dict_prop = {
        # Identifiers / metadata
        "UniProt ID": uniprot_id,
        "Gene Name": gene,
        "Mutation": mutation,
        "Wild": wild,
        "Mut": mut,
        "Pos": pos,
    }

    # Feature group: physicochemical properties (site + window + diffs + AA-class counts)
    add_physchem_features(dict_prop, wild, mut, win13)

    # Feature group: AAIndex mutation matrix deltas (N-term + C-term contexts)
    add_mutation_matrix_features(dict_prop, sequence, pos, wild, mut)

    # Feature group: local sequence motifs (for odds ratio)
    for k, v in motifs.items():
        dict_prop[k] = v

    # Feature group: odds ratio features (coded + raw)
    add_odds_ratio_features(dict_prop, motifs)

    # Feature group: network centrality
    add_network_features(dict_prop, df_network, wild, pos)

    # Feature group: PSSM features
    add_pssm_features(dict_prop, f4_lines, pos, wild, mut)

    # Feature group: AACon conservation features
    add_aacon_features(dict_prop, f_aacon, pos)

    # Feature group: disorder + DSSP ASA/SS + plDDT + residue depth
    add_disorder_depth_plddt_dssp(dict_prop, f_disorder, f_depth, f_plddt, f_dssp, pos)

    # Feature group: structure-specific interaction counts
    add_structure_interaction_counts(dict_prop, uniprot_id, pos)

    # Encode secondary structure to numeric (as in original)
    sec_map = {'B': 1, 'E': 2, 'H': 3, 'S': 4, 'T': 5, 'I': 6, '-': 0}
    dict_prop["sec_strc"] = sec_map.get(dict_prop.get("sec_strc", "-"), 0)

    all_rows.append(dict_prop)


# -----------------------------
# WRITE OUTPUT
# -----------------------------
all_prop = pd.DataFrame(all_rows)
all_prop.to_csv(OUT_CSV, index=False)
print(f"Wrote: {OUT_CSV}")

Wrote: ../output_data/complete_dataset_with_features.csv
